In [1]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
pagos_raw = pd.read_csv("../../data/raw/expensas_pagos.csv", 
                    sep=";", 
                    na_values = "NULL",
                    dtype = {
                        "id" : "object",
                        "expensa_id" :"object"
                    })

expensas_raw = pd.read_csv("../../data/raw/expensas.csv",
                        sep=";", 
                        na_values = "NULL",
                        dtype = {
                            "id": "object",
                            "unidad_id": "object",
                            "propietario_id": "object",
                            "inquilino_id": "object",
                            "consorcio_id": "object",
                            "expensa_padre_id": "object",
                            "unidad_prop": "object",
                            "consorcio_id": "object",
                            "usuario_creador_id": "object",
                            "concepto_id": "object",
                            "mes": "object",
                            "anio": "object",
                        })

consorcios_raw = pd.read_csv("../../data/raw/expensas_consorcios.csv",
                         sep = ";",
                         na_values = "NULL",
                         dtype = {
                            "id": "object",
                        })

unidades_raw = pd.read_csv("../../data/raw/expensas_unidades.csv",
                         sep = ";",
                         na_values = "NULL",
                         dtype = {
                            "id": "object",
                            "consorcio_id":"object",
                            "type_id":"object"
                         })

metodos_pago_raw = pd.read_csv("../../data/raw/expensas_pagos_metodos.csv",
                         sep = ";",
                         na_values = "NULL")

c:\users\smaldonado\documents\tfm\venv\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def remove_deleted(data):
    return data.loc[data.deleted == 0, :]

## Consorcios y Unidades

#### Consorcios

In [4]:
#Consorcios
consorcios = remove_deleted(consorcios_raw)
consorcios = consorcios[["id","name", "modoPago", "formaProrrateo", "tipoProrrateo"]]

#### Unidades

In [5]:
#Unidades
unidades = remove_deleted(unidades_raw)
unidades = unidades[["id","denomination","type_id","consorcio_id"]]
unidades = pd.merge(unidades,
                    consorcios,
                    left_on="consorcio_id",
                    right_on="id",
                    suffixes = ("","_consorcio"))
unidades = unidades.drop(columns="id_consorcio")
unidades.columns = ["unidad_id","unidad_denominacion", "unidad_tipo", "consorcio_id", "consorcio_nombre", "modo_pago", "forma_prorrateo", "tipo_prorrateo"]

## Expensas

In [48]:
expensas = remove_deleted(expensas_raw)
expensas = pd.merge(expensas,
                    unidades,
                    left_on=["consorcio_id","unidad_id"],
                    right_on=["consorcio_id","unidad_id"],
                    how="inner")

expensas_columns_to_delete = ['vencimiento_fijado',
                               'fecha_emision', 'saldo_utilizado_en_pago', 'monto_descuento', 'interes_pagado',
                               'monto_old', 'interes_pagado_old', 'monto_parcial_old',
                               'diferencia_pago', 'consorcio_admin', 'costo_eol',
                               'costo_comisionista', 'porcentaje_costo_eol',
                               'porcentaje_costo_comisionista', 'porcentaje_rapipago',
                               'porcentaje_pagomiscuentas','usuario_creador_id', 'deleted', 'unidad_denominacion_y',
                               'consorcio_nombre_y']

expensas = expensas.drop(columns=expensas_columns_to_delete)
expensas.columns = ['expensa_id', 'unidad_id', 'propietario_id', 'inquilino_id',
       'fecha_vencimiento_1', 'fecha_vencimiento_2', 'monto', 'monto_total',
       'monto_parcial', 'expensa_padre_id', 'int_dia', 'int_mes', 'mes',
       'anio', 'unidad_denominacion', 'unidad_prop', 'consorcio_id',
       'consorcio_nombre', 'version', 'concepto_id', 'unidad_tipo',
       'modo_pago', 'forma_prorrateo', 'tipo_prorrateo']

In [49]:
# Expenses
def fix_wrong_dates(expensas):
    
    vencido_12_selection = ((expensas.fecha_vencimiento_1 == '0000-00-00') 
                            | (expensas.fecha_vencimiento_2 == '0000-00-00')
                           | (expensas.fecha_vencimiento_1.str.contains('-00'))
                           | (expensas.fecha_vencimiento_2.str.contains('-00'))) & (expensas.modo_pago == 'mes_vencido') & (expensas.mes == 12)
    vencido_resto_selection = ((expensas.fecha_vencimiento_1 == '0000-00-00') | (expensas.fecha_vencimiento_2 == '0000-00-00') | (expensas.fecha_vencimiento_1.str.contains('-00'))
                           | (expensas.fecha_vencimiento_2.str.contains('-00'))) & (expensas.modo_pago == 'mes_vencido') & (expensas.mes != 12)
    adelantado_selection = ((expensas.fecha_vencimiento_1 == '0000-00-00') | (expensas.fecha_vencimiento_2 == '0000-00-00')| (expensas.fecha_vencimiento_1.str.contains('-00'))
                           | (expensas.fecha_vencimiento_2.str.contains('-00'))) & (expensas.modo_pago == 'mes_adelantado')
    

    #vencido_12_selection
    expensas.loc[vencido_12_selection, "fecha_vencimiento_1"] = (expensas.loc[vencido_12_selection, "anio"].map(int) + 1).map(str) + "-01-10"
    expensas.loc[vencido_12_selection, "fecha_vencimiento_2"] = (expensas.loc[vencido_12_selection, "anio"].map(int) + 1).map(str) + "-01-20"

    #vencido_resto_selection
    expensas.loc[vencido_resto_selection, "fecha_vencimiento_1"] = expensas.loc[vencido_resto_selection, "anio"] + "-" + (expensas.loc[vencido_resto_selection, "mes"].map(int) + 1).apply(lambda x: '{0:0>2}'.format(x)) + "-10"
    expensas.loc[vencido_resto_selection, "fecha_vencimiento_2"] = expensas.loc[vencido_resto_selection, "anio"] + "-" + (expensas.loc[vencido_resto_selection, "mes"].map(int) + 1).apply(lambda x: '{0:0>2}'.format(x)) + "-20"
    
    #adelantado_12_selection
    expensas.loc[adelantado_selection, "fecha_vencimiento_1"] = expensas.loc[adelantado_selection, "anio"] + "-" + expensas.loc[adelantado_selection, "mes"].apply(lambda x: '{0:0>2}'.format(x)) + "-10"
    expensas.loc[adelantado_selection, "fecha_vencimiento_2"] = expensas.loc[adelantado_selection, "anio"] + "-" + expensas.loc[adelantado_selection, "mes"].apply(lambda x: '{0:0>2}'.format(x)) + "-20"    
    
    return expensas
    
def filter_expenses_concepts(expensas, concept_list = ["1","10", "599"]):
    return expensas.loc[expensas.concepto_id.isin(concept_list),:]

def filter_expenses_debts(expensas):
    return expensas.loc[expensas.monto != expensas.monto_total,:]

def filter_negative_amounts(expensas):
    return expensas.loc[(expensas.monto > 0) & (expensas.monto_total > 0),:]

def filter_expenses(expensas):
    exp = fix_wrong_dates(expensas)
    exp = filter_expenses_concepts(exp)
    exp = filter_expenses_debts(exp)
    exp = filter_negative_amounts(exp)
    return exp

In [50]:
expensas = filter_expenses(expensas)

## Pagos

In [53]:
pagos = pd.merge(pagos_raw,
                 metodos_pago_raw,
                 left_on="metodo",
                 right_on="id",
                 suffixes = ("","_mp"))

pagos = pagos[["fecha","monto","expensa_id","nombre"]]
pagos.columns = ["fecha_pago", "monto_pago", "expensa_id", "metodo_pago"]

In [56]:
data = pd.merge(expensas,
                pagos,
                left_on="expensa_id",
                right_on="expensa_id",
                how="left")

In [57]:
data.shape

(264858, 27)

In [63]:
data = data.loc[data.anio.map(int)>=2014,:]

In [64]:
data.shape

(264541, 27)

In [70]:
data.columns

Index(['expensa_id', 'unidad_id', 'propietario_id', 'inquilino_id',
       'fecha_vencimiento_1', 'fecha_vencimiento_2', 'monto', 'monto_total',
       'monto_parcial', 'expensa_padre_id', 'int_dia', 'int_mes', 'mes',
       'anio', 'unidad_denominacion', 'unidad_prop', 'consorcio_id',
       'consorcio_nombre', 'version', 'concepto_id', 'unidad_tipo',
       'modo_pago', 'forma_prorrateo', 'tipo_prorrateo', 'fecha_pago',
       'monto_pago', 'metodo_pago'],
      dtype='object')

In [71]:
data.to_csv("../../data/processed/data.csv", index=False)